In [1]:
!pip install langchain
!pip install langchain-openai
!pip install langchainhub
!pip install pypdf
!pip install faiss-cpu

In [2]:
import os

# # Set OPENAI API Key

os.environ["OPENAI_API_KEY"] = "your openai key"
os.environ["TAVILY_API_KEY"] = "your tavily api key" # you can obtain your tavily api key here: https://tavily.com/
# OR (load from .env file)
# make sure you have python-dotenv installed
# from dotenv import load_dotenv
# load_dotenv("./.env")

Create your api key in the settings page of the [langsmith platform here](https://smith.langchain.com/settings).

In [3]:
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass() # obtain your langsmith api key here: https://smith.langchain.com/settings

> Summarized from this awesome video about the discussion of chains and agents: https://www.youtube.com/watch?v=bYLHklxEd_k&t=1801s


- Chains and agents represent two concepts in technology with distinct functionalities. Chains refer to linear sequences of events or processes that rely on the step-by-step execution of tasks, often requiring predefined inputs and producing predictable outcomes. 

- Agents, conversely, are autonomous systems capable of making decisions and performing tasks independently, adapting to new situations without explicit instructions. While chains excel in structured environments with clear rules, agents are designed to navigate complex, dynamic scenarios, learning and adjusting their behavior over time. The core difference lies in the level of autonomy and complexity each can handle, with agents typically seen as more advanced due to their self-governing nature.

Making a reporting agent.

Let's build an LLM agent that can build simple reports for tasks performed just by reading through the code and files inside a folder + notes from Notion page or .txt file.

First, let's organize our thoughts, an agent will be composed of:

- Tool
- Base LLM
- Conditions for stoping and returning an output 

![](2023-10-27-19-28-10.png)

In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
search = TavilySearchResults()

In [3]:
search.invoke("what is the weather in Portugal?")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Lisbon', 'region': 'Lisboa', 'country': 'Portugal', 'lat': 38.72, 'lon': -9.13, 'tz_id': 'Europe/Lisbon', 'localtime_epoch': 1718380622, 'localtime': '2024-06-14 16:57'}, 'current': {'last_updated_epoch': 1718379900, 'last_updated': '2024-06-14 16:45', 'temp_c': 23.2, 'temp_f': 73.8, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 17.4, 'wind_kph': 28.1, 'wind_degree': 320, 'wind_dir': 'NW', 'pressure_mb': 1017.0, 'pressure_in': 30.03, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 65, 'cloud': 25, 'feelslike_c': 25.1, 'feelslike_f': 77.1, 'windchill_c': 23.0, 'windchill_f': 73.3, 'heatindex_c': 25.0, 'heatindex_f': 76.9, 'dewpoint_c': 16.4, 'dewpoint_f': 61.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 6.0, 'gust_mph': 18.3, 'gust_kph': 29.4}}"},
 {'url': 'https://www.accuweather.com/en/pt/albufeira/273200/june-weather/2732

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction")
docs = loader.load()
docs

[Document(page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentLangChain 0.2 is out! Leave feedback on the v0.2 docs here. You can view the v0.1 docs here.IntegrationsAPI ReferenceMorePeopleContributingTemplatesCookbooks3rd party tutorialsYouTubearXivv0.2v0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector stores and retrieversBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use

In [5]:
docs[0]

Document(page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentLangChain 0.2 is out! Leave feedback on the v0.2 docs here. You can view the v0.1 docs here.IntegrationsAPI ReferenceMorePeopleContributingTemplatesCookbooks3rd party tutorialsYouTubearXivv0.2v0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector stores and retrieversBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use 

In [6]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

documents

[Document(page_content='Introduction | 🦜️🔗 LangChain', metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}),
 Document(page_content='Skip to main contentLangChain 0.2 is out! Leave feedback on the v0.2 docs here. You can view the v0.1 docs here.IntegrationsAPI ReferenceMorePeopleContributingTemplatesCookbooks3rd party tutorialsYouTubearXivv0.2v0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retr

In [7]:
vector = FAISS.from_documents(documents, OpenAIEmbeddings())

In [8]:
retriever = vector.as_retriever()

In [10]:
retriever.invoke("What is langchain?")[0]

Document(page_content='Introduction | 🦜️🔗 LangChain', metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'})

In [11]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langchain_search",
    "Search for information about LangChain. For any questions about LangChain, you must use this tool!",
)

In [12]:
tools = [search, retriever_tool]

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a34c74cb-118c-4748-ba84-947464b6dd41-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

In [14]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant'))

In [16]:
prompt.messages[1]

MessagesPlaceholder(variable_name='chat_history', optional=True)

In [17]:
prompt.messages[2]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))

In [18]:
prompt.messages[3]

MessagesPlaceholder(variable_name='agent_scratchpad')

In [19]:
tools

[TavilySearchResults(),
 Tool(name='langchain_search', description='Search for information about LangChain. For any questions about LangChain, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x12017cc20>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1240c5b50>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x12017cf40>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1240c5b50>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))]

In [20]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [21]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, 

In [22]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [23]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [24]:
agent_executor.invoke({"input": "What are some useful applications you can build with LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `langchain_search` with `{'query': 'useful applications of LangChain'}`


simplifies every stage of the LLM application lifecycle:Development: Build your applications using LangChain's open-source building blocks and components. Hit the ground running using third-party integrations and Templates.Productionization: Use LangSmith to inspect, monitor and evaluate your chains, so that you can continuously optimize and deploy with confidence.Deployment: Turn any chain into an API with LangServe.Concretely, the framework consists of the following open-source libraries:langchain-core: Base abstractions and LangChain Expression Language.langchain-community: Third party integrations.Partner packages (e.g. langchain-openai, langchain-anthropic, etc.): Some integrations have been further split into their own lightweight packages that only depend on langchain-core.langchain: Chains, agents, and retrieval strategies that make up an application's co

{'input': 'What are some useful applications you can build with LangChain?',
 'output': "LangChain can be used to build a variety of useful applications. Here are some applications you can build with LangChain:\n\n1. Development: Build applications using LangChain's open-source building blocks and components. Utilize third-party integrations and templates to kickstart your development process.\n\n2. Productionization: Use LangSmith to inspect, monitor, and evaluate your chains, enabling you to continuously optimize and deploy with confidence.\n\n3. Deployment: Turn any chain into an API with LangServe, allowing you to deploy your applications as REST APIs.\n\nLangChain framework consists of the following open-source libraries:\n- langchain-core: Base abstractions and LangChain Expression Language.\n- langchain-community: Third-party integrations.\n- Partner packages (e.g., langchain-openai, langchain-anthropic, etc.): Lightweight packages that depend on langchain-core.\n- langchain: Ch

Now we can inspect what is going on in the [langsmith platform!](https://smith.langchain.com/o/b68c89e6-dae0-54ab-a84b-3201d39a3c50/settings).